# Data visualization of stock daily tables

In [10]:
import numpy as np
import glob
import pandas as pd
from bokeh.plotting import figure, show, gridplot
from bokeh.io import output_notebook
from bokeh.models import HoverTool
from math import pi
output_notebook()

Loading BokehJS ...

### Function definition for plotting 

In [57]:
def plot_candlestick(in_df: pd.DataFrame, date: str, ma1: int = 0, ma2: int = 0) -> None:
    df = in_df.copy()
    w = 12*60*60*1000 # half day in ms
    if ma1 > 0:
        df['MA_'+str(ma1)] = df["<CLOSE>"].rolling(ma1).mean()
    if ma2 > 0:
        df['MA_'+str(ma2)] = df["<CLOSE>"].rolling(ma2).mean()
    
    df = df.loc[df.index >= date]
    inc = df["<CLOSE>"] > df["<OPEN>"]
    dec = df["<OPEN>"] > df["<CLOSE>"]
    TOOLS = "crosshair,pan,wheel_zoom,box_zoom,reset,save"

    pp = figure(x_axis_type="datetime", tools=TOOLS, plot_width=1000, title = df["<TICKER>"].values[0])
    ppv = figure(x_axis_type="datetime", x_range=pp.x_range, tools=TOOLS, plot_width=1000, plot_height=200, title="VOLUME")
    pp.xaxis.major_label_orientation = pi/4
    pp.xaxis.ticker.desired_num_ticks = 60
    pp.yaxis.ticker.desired_num_ticks = 20
    pp.grid.grid_line_alpha=0.3
    
    
    
    ppv.xaxis.major_label_orientation = pi/4
    ppv.xaxis.ticker.desired_num_ticks = 60
    ppv.yaxis.ticker.desired_num_ticks = 5
    ppv.grid.grid_line_alpha=0.3

    pp.segment(df.index, df["<HIGH>"], df.index, df["<LOW>"], color="black")
    pp.vbar(df.index[inc], w, df["<OPEN>"][inc], df["<CLOSE>"][inc], fill_color="#D5E1DD", line_color="black")
    pp.vbar(df.index[dec], w, df["<OPEN>"][dec], df["<CLOSE>"][dec], fill_color="#F2583E", line_color="black")
    pp.add_tools(HoverTool(tooltips=[( 'price',  '$y')]))
    if ma1 > 0:
        pp.line(df.index, df['MA_'+str(ma1)], color="blue")
    if ma2 > 0:
        pp.line(df.index, df['MA_'+str(ma2)], color="red")
    ppv.vbar(df.index, w, 0, df["<VOL>"], fill_color="#F2583E", line_color="black")
    
    gg = gridplot([[pp], [ppv]])
    
    show(gg)  # open a browser

## Find proper file based on symbol parameter

In [25]:
all_files = glob.glob(r"..\data\*.mst")
symbol = "XTB."
found_files = []
for path in all_files:
    if symbol in path:
        found_files.append(path)
found_files

['..\\data\\XTB.mst']

## Load data using pandas and plot it

In [26]:
my_data = pd.read_csv(found_files[0], index_col=1)
my_data.index = pd.to_datetime(my_data.index, format="%Y%m%d")
my_data

,<TICKER>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
<DTYYYYMMDD>,,,,,,
2016-05-17,XTB,13.14,13.14,12.95,13.00,405542
2016-05-18,XTB,13.11,13.70,13.11,13.58,123426
2016-05-19,XTB,13.30,13.90,13.20,13.90,53700
2016-05-20,XTB,13.90,14.30,13.65,14.30,55208
2016-05-23,XTB,14.25,14.25,13.60,14.00,36942
...,...,...,...,...,...,...
2020-05-08,XTB,8.70,8.98,8.44,8.70,2126452
2020-05-11,XTB,8.88,8.94,8.48,8.70,904281
2020-05-12,XTB,8.72,9.30,8.60,9.10,883032


In [24]:
date_start = "20191201"
plot_candlestick(my_data, date_start, 11, 15)

In [27]:
date_start = "20191201"
plot_candlestick(my_data, date_start, 11, 15)

### Bulk plotter

In [6]:
observed = ["CDPROJEKT", "CCC.", "11BIT", "XTB", "KGHM"]
all_files = glob.glob(r"..\data\*.mst")
path_list = []
for item in observed:
    for path in all_files:
        if item in path and "RCF" not in path:
            path_list.append(path)
path_list

['..\\data\\CDPROJEKT.mst',
 '..\\data\\CCC.mst',
 '..\\data\\11BIT.mst',
 '..\\data\\XTB.mst',
 '..\\data\\KGHM.mst']

In [59]:
path_basked = ['..\\data\\CDPROJEKT.mst',
               '..\\data\\CCC.mst',
               '..\\data\\11BIT.mst',
               '..\\data\\XTB.mst',
               '..\\data\\KGHM.mst']
date_start = "20191201"

for path in path_basked:
    data = pd.read_csv(path, index_col=1)
    data.index = pd.to_datetime(data.index, format="%Y%m%d")
    plot_candlestick(data, date_start, 11, 15)